## Projection and Inverse Kinematics

In [1]:
import sys
sys.path.append('lib')

import scipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time, os
from IPython.display import clear_output
from IPython.core import display

import pybullet as p
import pybullet_data
import pinocchio as pin

from pb_utils.kine import get_joint_limits, check_joint_limits
from pb_utils.visualize import set_q, vis_traj, create_primitives
from pin_utils.pin_utils import computeJacobian, computePose

from residuals import ResidualFrameTranslationFloatingBase, ResidualFrameRotationSE3FloatingBase, ResidualBound, \
                        ResidualPosture, ResidualSum, ResidualStructure, CostProjector, clip_bounds
from motion_planning import sampler
from panda_config import pb_joint_indices
from functools import partial

DATA_PATH = '../../data'
robot_urdf = DATA_PATH + '/urdf/panda_arm.urdf'

%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=True)

### Setup Pybullet

In [2]:
physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

In [3]:
p.resetSimulation()

### Setup Robot & environment

In [4]:
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
mean_pose = 0.5*(joint_limits[0]+joint_limits[1])

plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,0.), (0,0,0,1))

#for visualizing the desired target
_,_,ball_id = create_primitives(radius=0.05)

### Load model in pinocchio

In [5]:
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()

pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_hand2')

#### Define standard functions

In [11]:
computeJacobian_std = partial(computeJacobian, rmodel, rdata, ee_frame_id)
computePose_std = partial(computePose, rmodel, rdata, ee_frame_id)
set_q_std = partial(set_q,robot_id = robot_id, joint_indices=pb_joint_indices)

***

## Using Cost Model for projection

In [7]:
rotation_ik = np.eye(3) #desired orientation matrix
pose_cost = ResidualFrameTranslationFloatingBase(rmodel, rdata, np.zeros(3), ee_frame_id, weight=np.array([1.,1.,1.]))
ori_cost = ResidualFrameRotationSE3FloatingBase(rmodel, rdata, rotation_ik, ee_frame_id, weight=np.array([1.,1.,0.]))
bound_cost = ResidualBound(joint_limits, 1e-4)
posture_cost = ResidualPosture(rmodel, rdata, mean_pose)

In [8]:
#for projection
cost_sum = ResidualSum(rmodel, rdata)
cost_sum.addCost(ori_cost, 20., 'ori_cost', 1e-4)
cost_sum.addCost(bound_cost, 20., 'joint_limit', 1e-3)

#for IK with horizontal gripper constraint
cost_sum_ik = ResidualSum(rmodel, rdata)
cost_sum_ik.addCost(pose_cost,50., 'pose_cost', 1e-3)
cost_sum_ik.addCost(ori_cost, 30., 'ori_cost', 1e-3)
cost_sum_ik.addCost(bound_cost, 30., 'joint_limit', 1e-3)
# cost_sum_ik.addCost(posture_cost, .001 , 'posture', 1e3)

robot_projector = CostProjector(cost_sum, rmodel, rdata, bounds = joint_limits)
robot_ik_solver = CostProjector(cost_sum_ik, rmodel, rdata, bounds = joint_limits)

#### Define the sampler

In [9]:
workspace_area = np.array([[-0.6, -0.6 , -0.1], [0.6, 0.6, 1.0]])
target_sampler = sampler(workspace_area)
joint_limits_margin = np.copy(joint_limits)
joint_limits_margin[:,0] -= 0.2
joint_limits_margin[:,1] += 0.2
rob_simple_sampler = sampler(joint_limits_margin)

#### Try the projection operator

In [14]:
q = rob_simple_sampler.sample().flatten()
res = robot_projector.project(q)
q, success, func_calls = res['q'], res['stat'], res['nfev']
set_q_std(q)

print(robot_projector.cost.calc_cost(q))

1.597409283487189e-11


#### Compare IK

In [15]:
ik_workspace_area = np.array([[0.4, -0.4 , 0.5], [0.5, 0.5, .8]])
ik_target_sampler = sampler(ik_workspace_area)
goal_poses = ik_target_sampler.sample(1000)
for i in range(len(goal_poses)):
    goal_poses[i] = clip_bounds(goal_poses[i], ik_workspace_area)

#### Try IK

In [40]:
idx = np.random.randint(1000)
goal_pos = goal_poses[idx]
    
p.resetBasePositionAndOrientation(ball_id, goal_pos, np.array([0,0,0,1]))

robot_ik_solver.cost.costs['pose_cost'].cost.desired_pose = goal_pos

q0 = rob_simple_sampler.sample().flatten()
set_q_std(q0)


robot_ik_solver.update_pinocchio(q0)

print(robot_ik_solver.cost.calc_cost(q0))
res = robot_ik_solver.project(q0, maxiter = 300)
print('Status:', res['stat'], ', Number of function calls: ', res['nfev'])
set_q_std(res['q'])
print('Cost: ', robot_ik_solver.cost.calc_cost(res['q']))

robot_ik_solver.check_config(res['q'])

1902.0936511826253
Status: True , Number of function calls:  14
Cost:  2.1882732400119846e-07

 CHECKING
Residual names:
pose_cost,ori_cost,joint_limit,

Residual values:
[-0. -0. -0.],[ 0.  0. -0.],[0. 0. 0. 0. 0. 0. 0.],

Jacobian SVD:
[52.3832 43.1388 31.5384 27.6716 11.2849  8.5156 -0.    ]

Next direction:
[ 0. -0. -0. -0.  0. -0.  0.]
